In [9]:
import os
import datetime as dt
import pandas as pd

In [10]:
repo_dir = "/Users/ameliachu/repos/nlu-reddit-toxicity-dataset"
labelled_data_dir = f"{repo_dir}/data/labelled/"
labelled_data_fnames = [f for f in os.listdir(labelled_data_dir)]
master_data_location  = f"{repo_dir}/data/labelled_master_data_{dt.date.today()}.csv"
print(labelled_data_fnames)
print(master_data_location)

['yj2369_labelling_assignment_2022-04-13.csv', 'gm2858_labelling_assignment_2022-04-21.csv', 'gm2858_labelling_assignment_2022-04-09.csv', 'yp2201_labelling_assignment_2022-04-09.csv', 'yp2201_labelling_assignment_2022-04-21.csv', 'ac4119_labelling_assignment_2022-04-09.csv', 'ac4119_labelling_assignment_2022-04-21.csv', 'ac4119_labelling_assignment_2022-04-13.csv', 'yp2201_labelling_assignment_2022-04-13.csv', 'gm2858_labelling_assignment_2022-04-13.csv', 'yj2369_labelling_assignment_2022-04-09.csv', 'yj2369_labelling_assignment_2022-04-21.csv']
/Users/ameliachu/repos/nlu-reddit-toxicity-dataset/data/labelled_master_data_2022-04-28.csv


In [11]:
selected_columns = ['example_id', 'preceding_comment', 'comment_for_evaluation',
       'following_comment', 'toxicity', 'severe_toxicity', 'identity_attack',
       'insult', 'profanity', 'threat']

list_of_labelled_data = [pd.read_csv(f"{labelled_data_dir}{fname}")[selected_columns] for fname in labelled_data_fnames]

In [12]:
selected_columns = ['example_id', 'preceding_comment', 'comment_for_evaluation',
       'following_comment', 'toxicity', 'severe_toxicity', 'identity_attack',
       'insult', 'profanity', 'threat']

list_of_labelled_data = [pd.read_csv(f"{labelled_data_dir}{fname}")[selected_columns] for fname in labelled_data_fnames]

In [13]:
labelled_master_data = pd.concat(list_of_labelled_data ).drop_duplicates()

In [14]:
labelled_master_data.to_csv(master_data_location,index=False)

In [15]:
labelled_master_data.head(5)

,example_id,preceding_comment,comment_for_evaluation,following_comment,toxicity,severe_toxicity,identity_attack,insult,profanity,threat
0,102693,It’s not ready yet,Margin? I hardly knew her,"Fucking JPow, you were supposed to help the bu...",0.0,0.0,0.0,0.0,0.0,0.0
1,115299,did wsb pump something today? looking for a pu...,SL GG get in now,"What abt all the March ones I have, can they b...",0.0,0.0,0.0,0.0,0.0,0.0
2,120753,We're confiscating this thread\n\nOfficial GME...,DO NOT put respek on that man's name by callin...,?,0.0,0.0,0.0,0.0,0.0,0.0
3,128106,Have been hoping for 2 green days in a row for...,T'is but a scratch - Elon,BNGO,0.0,0.0,0.0,0.0,0.0,0.0
4,133758,AAPL is the most confusing part of my portfoli...,Those were the best. \nCan we blast Boomer kin...,BB hitting $10 again this week.,0.0,0.0,0.0,0.0,0.0,1.0


In [16]:
len(labelled_master_data)

2017